<center><h1>PUBG Player's Data Analysis and Placement Prediction</h1></center>
<center><h2>Fan Wang &nbsp;&nbsp;Bolun Yan </h2></center>
<center><h2>A53277514 &nbsp;&nbsp;A92413094 </h2></center>

In [1]:
# Preparation
import numpy as py
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

<h1>Part 1 - Data Pre-process </h1>

In [2]:
train = pd.read_csv('data/train_v2.csv')

In [3]:
train.cloums()

AttributeError: 'DataFrame' object has no attribute 'cloums'

<h1> Part 2 - Data Visualization and Analysis </h1>

<h1> Part 3 - Placement Prediction </h1>